# MAUP (problema da unidade de área modificável)

Além da dependência espacial e da heterogeneidade espacial (os efeitos espaciais), existem alguns problemas que podem afetar a análise de dados espaciais:

- falácia ecológica
- unidade de área modificável (MAUP) - sensibilidade dos resultados em relação à dimensão e à configuração da área de estudo
- efeito de beirada
- influência de *outliers* espaciais

Os resultados de uma análise de dados dependem da definição do critério usado para a agregação espacial dos dados.




---



---


MAUP = problema de escala + problema de zoneamento

---





- O MAUP se manifesta na situação em que os resultados da análise são sensíveis à forma como os dados espaciais são organizados (nível de escala e zoneamento)
- Uma escolha inadequada da escala geográfica pode gerar dependência espacial artificial
- O MAUP limita a possibilidade de replicação de um modelo a uma outra região de estudo, se a escala e o zoneamento forem distintos daqueles da aplicação inicial

- Se os dados fossem distribuídos aleatoriamente no espaço, o problema de escala não apareceria
- O problema de escala vincula-se ao efeito da dependência espacial
- Se o espaço fosse absolutamente homogêneo, qualquer combinação de unidades espaciais em zonas forneceria os mesmos resultados (não haveria o problema de zoneamento)

# MAUP em MG

In [ ]:
!pip install geopandas==0.8.2
!pip install --upgrade pyshp
!pip install shapely  ==1.7.0
!pip install --upgrade descartes
!pip install mapclassify==2.3.0 libpysal==4.3.0 splot==1.1.3
!pip install esda
!pip install pysal

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as sm

# para gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# para a análise de dados espaciais
import geopandas as gp
import pysal as ps
import splot
import mapclassify as mc
from libpysal.weights import Queen
from libpysal import weights
from esda import Moran, Moran_Local, G_Local
from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation

# links com arquivos úteis
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/'
link_p = 'https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/'

# Nível de agregação: municípios

## Quantidade produzida

Dados de 2017

In [ ]:
# ler dados de interesse
dados = pd.read_csv('https://raw.githubusercontent.com/patriciasiqueira/patriciasiqueira.github.io/master/arquivos/qtde-cod-18-mg.csv', encoding='latin1')
link = 'https://raw.githubusercontent.com/lincolnfrias/dados/master/mg.json'
geodf = gp.read_file(link)   # ler shapefile de MG
# mesclar shapefile com dataframe usando a coluna mun
geodf.rename(columns={'CD_GEOCMU': 'mun'}, inplace=True)  # mudar nome da coluna com cód. para 'mun'
geodf['mun'] = geodf.mun.astype(int)  # transformar códigos em inteiros
mg = pd.merge(geodf, dados, on='mun', suffixes=('', '_y'))  # mesclar o dataframe e o shapefile em um objeto
mg = mg.loc[:, mg.columns != 'NM_MUNICIP']  # tirar a coluna NM_MUNICIP

In [ ]:
mg.columns

In [ ]:
mg.query("nome_mun == 'Varginha'")

In [ ]:
variavel = 'banana'      # escolher a variável e colocar entre aspas

In [ ]:
# total da produção de banana em MG em 2017
mg[variavel].sum()

In [ ]:
# resumo estatístico
mg[variavel].describe()

In [ ]:
# histograma
mg[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(mg[variavel], orient='v');

In [ ]:
# visualizar os municípios em ordem decrescente (ou crescente)
mg.sort_values(variavel, ascending=False)[['nome_mun', 'nome_meso', variavel]].head(10)

In [ ]:
mg[mg.nome_mun == "Jaíba"]

In [ ]:
# mapa temático - simples
mg.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
mg.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
mg.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(mg)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = mg[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, mg, p=0.05, figsize = (9,9));

# Nível de agregação: mesorregiões

In [ ]:
# agrupar municípios em mesorregiões
# mapa contém as informações divididas em municípios e queremos unir os
# municípios em mesorregiões
# uso da função dissolve
# agrupar de acordo com as mesorregiões (código e nome)
# somar os valores dos municípios presentes em cada mesorregião
mesos = mg.dissolve(by=['meso', 'nome_meso'], aggfunc='sum')
mesos.head(3)

In [ ]:
# reiniciar índices e retirar as colunas que não serão usadas
mesos = mesos.reset_index()
# retirar = ['mun', 'uf', 'micro', 'cod_rgi', 'nome_rgi', 'nome_rgint', 'codmun6', 'regiao', 'nome_regiao', 'area']
# mesos = mesos.drop(retirar, axis=1)

In [ ]:
# resumo estatístico
mesos[variavel].describe()

In [ ]:
mesos[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(mesos[variavel], orient='v');

In [ ]:
# visualizar as mesorregiões em ordem decrescente (ou crescente)
mesos.sort_values(variavel, ascending=False)[['nome_meso', variavel]].head(10)

In [ ]:
# mapa temático - simples
mesos.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
mesos.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
mesos.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(mesos)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = mesos[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, mesos, p=0.05, figsize = (9,9));

# Nível de agregação: microrregiões

In [ ]:
micros = mg.dissolve(by=['micro', 'nome_micro'], aggfunc='sum')
micros.head(3)

In [ ]:
# reiniciar índices
micros = micros.reset_index()

In [ ]:
# resumo estatístico
micros[variavel].describe()

In [ ]:
micros[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(micros[variavel], orient='v');

In [ ]:
# visualizar as regiões em ordem decrescente (ou crescente)
micros.sort_values(variavel, ascending=False)[['nome_micro', variavel]].head(10)

In [ ]:
# mapa temático - simples
micros.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
micros.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
micros.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(micros)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = micros[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, micros, p=0.05, figsize = (9,9));

# Nível de agregação: regiões intermediárias

In [ ]:
mg.columns

In [ ]:
# agrupar municípios em regiões intermediárias
# mapa contém as informações divididas em municípios e queremos unir os
# municípios em regiões intermediárias
# uso da função dissolve
# somar os valores dos municípios presentes em cada região intermediária
rgints = mg.dissolve(by=['cod_rgint', 'nome_rgint'], aggfunc='sum')
rgints.head(3)

In [ ]:
# reiniciar índices e retirar as colunas que não serão usadas
rgints = rgints.reset_index()

In [ ]:
# resumo estatístico
rgints[variavel].describe()

In [ ]:
rgints[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(rgints[variavel], orient='v');

In [ ]:
# visualizar as regiões em ordem decrescente (ou crescente)
rgints.sort_values(variavel, ascending=False)[['nome_rgint', variavel]].head(10)

In [ ]:
# mapa temático - simples
rgints.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
rgints.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
rgints.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(rgints)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = rgints[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, rgints, p=0.05, figsize = (9,9));

# Nível de agregação: regiões imediatas

In [ ]:
mg.columns

In [ ]:
# somar os valores dos municípios presentes em cada região imediata
rgis = mg.dissolve(by=['cod_rgi', 'nome_rgi'], aggfunc='sum')
rgis.head(3)

In [ ]:
# reiniciar índices
rgis = rgis.reset_index()

In [ ]:
# resumo estatístico
rgis[variavel].describe()

In [ ]:
rgis[variavel].hist();

In [ ]:
# boxplot
sns.boxplot(rgis[variavel], orient='v');

In [ ]:
# visualizar as regiões em ordem decrescente (ou crescente)
rgis.sort_values(variavel, ascending=False)[['nome_rgi', variavel]].head(10)

In [ ]:
# mapa temático - simples
rgis.plot(variavel, legend=True);

In [ ]:
# mapa temático - quantiles
rgis.plot(column=variavel, figsize=(10, 10), scheme='Quantiles', cmap='OrRd', legend=True, k=4);

In [ ]:
# mapa temático - equal_interval
rgis.plot(column=variavel, figsize=(10, 10), scheme='Equal_Interval', cmap='OrRd', legend=True, k=4);

In [ ]:
# obter matriz de vizinhança no formato queen
w = Queen.from_dataframe(rgis)
w.transform = 'r'
# calcular I de Moran global para a variável escolhida
y = rgis[variavel].values
moran = Moran(y, w)
moran.I

In [ ]:
# valor-p
moran.p_sim

In [ ]:
# só o diagrama de dispersão
moran_loc = Moran_Local(y, w)
moran_scatterplot(moran_loc, p=0.05);

In [ ]:
# mapa LISA
lisa_cluster(moran_loc, rgis, p=0.05, figsize = (9,9));

# Apenas LISA em diferentes níveis de agregação

In [ ]:
results = []

In [ ]:
w = Queen.from_dataframe(mg)
w.transform = 'r'
y = mg[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, mg, p=0.05, figsize = (9,9));
print(moran.I, moran.p_sim)
results.append([moran.I, moran.p_sim])

In [ ]:
w = Queen.from_dataframe(mesos)
w.transform = 'r'
y = mesos[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, mesos, p=0.05, figsize = (9,9));
print(moran.I, moran.p_sim)
results.append([moran.I, moran.p_sim])

In [ ]:
w = Queen.from_dataframe(micros)
w.transform = 'r'
y = micros[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, micros, p=0.05, figsize = (9,9));
print(moran.I, moran.p_sim)
results.append([moran.I, moran.p_sim])

In [ ]:
w = Queen.from_dataframe(rgints)
w.transform = 'r'
y = rgints[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, rgints, p=0.05, figsize = (9,9));
print(moran.I, moran.p_sim)
results.append([moran.I, moran.p_sim])

In [ ]:
w = Queen.from_dataframe(rgis)
w.transform = 'r'
y = rgis[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)
lisa_cluster(moran_loc, rgis, p=0.05, figsize = (9,9));
print(moran.I, moran.p_sim)
results.append([moran.I, moran.p_sim])

In [ ]:
results

In [ ]:
# dicionário de dataframes
niveis = {
  'mg': mg,
  'mesos': mesos,
  'micros': micros,
  'rgints': rgints,
  'rgis': rgis
}

In [ ]:
for k in niveis.keys():
    w = Queen.from_dataframe(niveis[k])
    w.transform = 'r'
    y = niveis[k][variavel].values
    moran = Moran(y, w)
    moran_loc = Moran_Local(y, w)
    lisa_cluster(moran_loc, niveis[k], p=0.05, figsize = (9,9));

In [ ]:
results

In [ ]:
# dicionário de dataframes
niveis = {
  'mg': mg,
  'mesos': mesos,
  'micros': micros,
  'rgints': rgints,
  'rgis': rgis
}

In [ ]:
for k in niveis.keys():
    w = Queen.from_dataframe(niveis[k])
    w.transform = 'r'
    y = niveis[k][variavel].values
    moran = Moran(y, w)
    moran_loc = Moran_Local(y, w)
    lisa_cluster(moran_loc, niveis[k], p=0.05, figsize = (9,9));

# Inclusão dos nomes - LISA

In [ ]:
# para evitar problemas com os nomes: https://gis.stackexchange.com/questions/372564/userwarning-when-trying-to-get-centroid-from-a-polygon-geopandas
mg = mg.to_crs(epsg=3395)
mesos = mesos.to_crs(epsg=3395)
micros = micros.to_crs(epsg=3395)
rgints = rgints.to_crs(epsg=3395)
rgis = rgis.to_crs(epsg=3395)

In [ ]:
# municípios
w = Queen.from_dataframe(mg)
w.transform = 'r'
y = mg[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)

fig, ax = plt.subplots(1, figsize=(15, 15))
lisa_cluster(moran_loc, mg, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05
posicoes = np.where(sig)
escolhidos = mg.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1],
         escolhidos.nome_mun[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')

In [ ]:
w = Queen.from_dataframe(mesos)
w.transform = 'r'
y = mesos[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)

fig, ax = plt.subplots(1, figsize=(15, 15))
lisa_cluster(moran_loc, mesos, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05
posicoes = np.where(sig)
escolhidos = mesos.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1],
         escolhidos.nome_meso[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')

In [ ]:
w = Queen.from_dataframe(micros)
w.transform = 'r'
y = micros[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)

fig, ax = plt.subplots(1, figsize=(15, 15))
lisa_cluster(moran_loc, micros, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05
posicoes = np.where(sig)
escolhidos = micros.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1],
         escolhidos.nome_micro[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')

In [ ]:
w = Queen.from_dataframe(rgints)
w.transform = 'r'
y = rgints[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)

fig, ax = plt.subplots(1, figsize=(15, 15))
lisa_cluster(moran_loc, rgints, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05
posicoes = np.where(sig)
escolhidos = rgints.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1],
         escolhidos.nome_rgint[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')

In [ ]:
w = Queen.from_dataframe(rgis)
w.transform = 'r'
y = rgis[variavel].values
moran = Moran(y, w)
moran_loc = Moran_Local(y, w)

fig, ax = plt.subplots(1, figsize=(15, 15))
lisa_cluster(moran_loc, rgis, p=0.05, ax=ax, figsize = (9,9));
sig = moran_loc.p_sim < 0.05
posicoes = np.where(sig)
escolhidos = rgis.iloc[posicoes[0]]
for j in escolhidos.index:
    ax.text(escolhidos.geometry.centroid[j].coords[0][0], escolhidos.geometry.centroid[j].coords[0][1],
         escolhidos.nome_rgi[j],
         fontsize=10, horizontalalignment='center', verticalalignment='bottom')